In [1]:
import sys
sys.path.append('..')
import abd_profile
import GCN
import warnings
import pandas as pd
import os

# avoid log warning
warnings.filterwarnings('ignore')

ori_profile = abd_profile.input_profile('../data/NAFLD/abd.tsv' , transfer=True)
ori_GCN = GCN.input_GCN('../data/NAFLD/NASH_GCN.tsv', transfer=True)
print(ori_profile.shape)
print(ori_GCN.shape)
metadata = pd.read_csv('../data/NAFLD/NASH_forward_63_map.txt', sep = '\t', header=0, index_col=0)


(63, 97)
(4324, 97)


In [2]:
# compute the distance
d_df = GCN.sp_d(ori_GCN)
params = {
    'sample cluster': 'enterotype', # 'enterotype', # can be 'seat'
    'max_cluster': 10,
    'max_depth': 6
}
d_df.to_csv('../data/NAFLD/NASH_distance.tsv', sep = '\t')

In [3]:
import copy
import analysis
# a new added block with packet function
outdir = '../result/NAFLD'
if not os.path.exists(outdir):
    os.mkdir(outdir)
# split samples according to phenotype
cluster_labels = list(set(metadata['DiseaseStatus']))
label_dict = {}
cluster_profiles = {}
phenos = ['NASH', 'Normal']
for pheno in phenos:
    idx_list = list(metadata[metadata['DiseaseStatus'] == pheno].index)
    cluster_profiles[pheno] = copy.deepcopy(ori_profile.loc[idx_list, :])

result, c = analysis.main(ori_GCN, ori_profile, outdir, params, d_df, cluster_profiles)


<Figure size 1296x1080 with 0 Axes>

<Figure size 1296x1080 with 0 Axes>

In [4]:
# check the difference
sp_dict  = {}
for pheno in phenos:
    cdir = os.path.join(outdir, 'cluster_{}'.format(pheno))
    eigen_path = os.path.join(cdir, 'eigen_node.tsv')
    eigen_df = pd.read_csv(eigen_path, sep='\t', header=0, index_col=0)
    eigen_cluster  = (eigen_df[(eigen_df['is_eigen'] == True) & (eigen_df['layer'] == 1)])['leaves'].iloc[0].split(',')
    sp_dict[pheno] = copy.deepcopy(set(eigen_cluster))

common = sp_dict[phenos[0]].intersection(sp_dict[phenos[1]])
diff0 = sp_dict[phenos[0]].difference(sp_dict[phenos[1]])
diff1 = sp_dict[phenos[1]].difference(sp_dict[phenos[0]])

print('Common: {} species: \n{}\n'.format(len(common), '\n'.join(sorted(list(common)))))
print('In {} only: {} species: \n{}\n'.format(len(diff0), phenos[0], '\n'.join(sorted(list(diff0)))))
print('In {} only: {} species: \n{}\n'.format(len(diff1), phenos[1], '\n'.join(sorted(list(diff1)))))


Common: 6 species: 
s__A. ruminis-OTU75
s__B. barnesiae
s__B. caecicola-OTU4
s__B. producta-OTU32
s__F. prausnitzii-OTU8
s__F. prausnitzii-OTU80

In 15 only: NASH species: 
s__A. octavius
s__B. coprophilus
s__C. pinnipediorum
s__D. pneumosintes
s__E. peruensis
s__Escherichia-Shigella-OTU88
s__Ezakiella-OTU94
s__F. magna
s__P. buccalis
s__P. ivorii
s__P. lacrimalis
s__P. loveana
s__P. olsenii
s__P. sp. 2007b
s__S5-A14a-OTU76

In 19 only: Normal species: 
s__A. caccae
s__A. indistinctus-OTU57
s__B. caecicola-OTU51
s__B. coprosuis-OTU67
s__B. hydrogenotrophica
s__B. magnum
s__B. producta-OTU81
s__C. comes
s__D. formicigenerans
s__D. longicatena
s__E. oxidoreducens
s__F. saccharivorans
s__P. chartae-OTU69
s__R. gnavus
s__R. torques
s__Ruminiclostridium-OTU96
s__Ruminococcaceae-OTU46
s__S. variabile-OTU44
s__S. variabile-OTU93

